**Prediction of Sea Surface Temperature for evaluation.csv**

*Importing Libraries*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

*Reading Files*

In [2]:
train = pd.read_csv('train.csv')
evaluation = pd.read_csv('evaluation.csv')

In [3]:
train

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.
0,1992,12,13,-4.99,156.00,4.8,-2.8,78.5,28.98,30.06
1,1989,5,14,0.00,-110.10,-2.5,2.4,NaN,24.95,25.23
2,1993,6,28,2.04,-110.20,-4.5,5.9,NaN,25.78,27.84
3,1995,1,8,1.98,-140.00,-7.7,1.2,82.6,26.47,26.86
4,1995,2,21,5.01,-170.00,-5.4,-6.6,75.2,28.83,28.90
...,...,...,...,...,...,...,...,...,...,...
104404,1989,10,17,9.04,-140.20,NaN,NaN,NaN,NaN,27.80
104405,1995,10,20,-5.00,-180.00,-3.2,-2.9,73.8,28.31,29.44
104406,1984,11,18,0.06,-110.06,-4.0,3.2,NaN,NaN,21.10
104407,1996,12,22,-0.01,-155.00,-6.9,-2.1,78.1,25.81,25.95


In [4]:
evaluation

,Unnamed: 0,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.
0,0,1995,6,22,2.00,165.0,-3.3,-1.9,79.4,28.95
1,1,1995,1,13,4.99,156.0,-3.5,-2.2,82.0,27.86
2,2,1995,7,9,-4.97,-125.0,-8.2,1.5,84.2,26.02
3,3,1996,7,17,8.01,165.0,-5.4,-1.0,84.1,28.38
4,4,1993,8,23,2.06,165.0,2.0,0.6,75.4,28.47
...,...,...,...,...,...,...,...,...,...,...
26098,26098,1996,3,4,8.00,156.0,-6.3,-2.6,79.6,27.58
26099,26099,1994,11,25,2.01,-179.9,-3.5,-2.6,78.3,29.97
26100,26100,1994,7,16,5.00,-170.0,-4.7,1.2,76.6,28.60
26101,26101,1993,7,12,-0.04,-170.0,-1.6,2.3,80.6,28.66


In [6]:
train.reset_index()
evaluation.reset_index()
train['Index']=train.index
evaluation['Index']=evaluation.index
evaluation.drop(columns=['Unnamed: 0'],inplace=True)

In [7]:
train

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.,Index
0,1992,12,13,-4.99,156.00,4.8,-2.8,78.5,28.98,30.06,0
1,1989,5,14,0.00,-110.10,-2.5,2.4,NaN,24.95,25.23,1
2,1993,6,28,2.04,-110.20,-4.5,5.9,NaN,25.78,27.84,2
3,1995,1,8,1.98,-140.00,-7.7,1.2,82.6,26.47,26.86,3
4,1995,2,21,5.01,-170.00,-5.4,-6.6,75.2,28.83,28.90,4
...,...,...,...,...,...,...,...,...,...,...,...
104404,1989,10,17,9.04,-140.20,NaN,NaN,NaN,NaN,27.80,104404
104405,1995,10,20,-5.00,-180.00,-3.2,-2.9,73.8,28.31,29.44,104405
104406,1984,11,18,0.06,-110.06,-4.0,3.2,NaN,NaN,21.10,104406
104407,1996,12,22,-0.01,-155.00,-6.9,-2.1,78.1,25.81,25.95,104407


In [8]:
evaluation

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,Index
0,1995,6,22,2.00,165.0,-3.3,-1.9,79.4,28.95,0
1,1995,1,13,4.99,156.0,-3.5,-2.2,82.0,27.86,1
2,1995,7,9,-4.97,-125.0,-8.2,1.5,84.2,26.02,2
3,1996,7,17,8.01,165.0,-5.4,-1.0,84.1,28.38,3
4,1993,8,23,2.06,165.0,2.0,0.6,75.4,28.47,4
...,...,...,...,...,...,...,...,...,...,...
26098,1996,3,4,8.00,156.0,-6.3,-2.6,79.6,27.58,26098
26099,1994,11,25,2.01,-179.9,-3.5,-2.6,78.3,29.97,26099
26100,1994,7,16,5.00,-170.0,-4.7,1.2,76.6,28.60,26100
26101,1993,7,12,-0.04,-170.0,-1.6,2.3,80.6,28.66,26101


*Filling missing values using method of multiple imputation*

In [9]:
X = train[train.columns].values
Y = evaluation[evaluation.columns].values
imputer = IterativeImputer(max_iter=10, random_state=0)
X = imputer.fit_transform(X)
Y = imputer.fit_transform(Y)
train[train.columns]=X
evaluation[evaluation.columns]=Y

In [10]:
train

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.,Index
0,1992.0,12.0,13.0,-4.99,156.00,4.800000,-2.800000,78.500000,28.980000,30.06,0.0
1,1989.0,5.0,14.0,0.00,-110.10,-2.500000,2.400000,87.181618,24.950000,25.23,1.0
2,1993.0,6.0,28.0,2.04,-110.20,-4.500000,5.900000,83.553169,25.780000,27.84,2.0
3,1995.0,1.0,8.0,1.98,-140.00,-7.700000,1.200000,82.600000,26.470000,26.86,3.0
4,1995.0,2.0,21.0,5.01,-170.00,-5.400000,-6.600000,75.200000,28.830000,28.90,4.0
...,...,...,...,...,...,...,...,...,...,...,...
104404,1989.0,10.0,17.0,9.04,-140.20,-3.119347,1.463338,84.453973,26.772222,27.80,104404.0
104405,1995.0,10.0,20.0,-5.00,-180.00,-3.200000,-2.900000,73.800000,28.310000,29.44,104405.0
104406,1984.0,11.0,18.0,0.06,-110.06,-4.000000,3.200000,92.976503,21.138393,21.10,104406.0
104407,1996.0,12.0,22.0,-0.01,-155.00,-6.900000,-2.100000,78.100000,25.810000,25.95,104407.0


In [11]:
evaluation

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,Index
0,1995.0,6.0,22.0,2.00,165.0,-3.3,-1.9,79.400000,28.95,0.0
1,1995.0,1.0,13.0,4.99,156.0,-3.5,-2.2,82.000000,27.86,1.0
2,1995.0,7.0,9.0,-4.97,-125.0,-8.2,1.5,84.200000,26.02,2.0
3,1996.0,7.0,17.0,8.01,165.0,-5.4,-1.0,84.100000,28.38,3.0
4,1993.0,8.0,23.0,2.06,165.0,2.0,0.6,75.400000,28.47,4.0
...,...,...,...,...,...,...,...,...,...,...
26098,1996.0,3.0,4.0,8.00,156.0,-6.3,-2.6,79.600000,27.58,26098.0
26099,1994.0,11.0,25.0,2.01,-179.9,-3.5,-2.6,78.300000,29.97,26099.0
26100,1994.0,7.0,16.0,5.00,-170.0,-4.7,1.2,76.600000,28.60,26100.0
26101,1993.0,7.0,12.0,-0.04,-170.0,-1.6,2.3,80.600000,28.66,26101.0


In [12]:
train['Date'] = pd.to_datetime(train[['year','month','day']])
train.set_index(train['Date'],inplace=True)
train.drop(columns=['Date'],inplace=True)
evaluation['Date'] = pd.to_datetime(evaluation[['year','month','day']])
evaluation.set_index(evaluation['Date'],inplace=True)
evaluation.drop(columns=['Date'],inplace=True)

In [13]:
train

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.,Index
Date,,,,,,,,,,,
1992-12-13,1992.0,12.0,13.0,-4.99,156.00,4.800000,-2.800000,78.500000,28.980000,30.06,0.0
1989-05-14,1989.0,5.0,14.0,0.00,-110.10,-2.500000,2.400000,87.181618,24.950000,25.23,1.0
1993-06-28,1993.0,6.0,28.0,2.04,-110.20,-4.500000,5.900000,83.553169,25.780000,27.84,2.0
1995-01-08,1995.0,1.0,8.0,1.98,-140.00,-7.700000,1.200000,82.600000,26.470000,26.86,3.0
1995-02-21,1995.0,2.0,21.0,5.01,-170.00,-5.400000,-6.600000,75.200000,28.830000,28.90,4.0
...,...,...,...,...,...,...,...,...,...,...,...
1989-10-17,1989.0,10.0,17.0,9.04,-140.20,-3.119347,1.463338,84.453973,26.772222,27.80,104404.0
1995-10-20,1995.0,10.0,20.0,-5.00,-180.00,-3.200000,-2.900000,73.800000,28.310000,29.44,104405.0
1984-11-18,1984.0,11.0,18.0,0.06,-110.06,-4.000000,3.200000,92.976503,21.138393,21.10,104406.0


In [14]:
evaluation

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,Index
Date,,,,,,,,,,
1995-06-22,1995.0,6.0,22.0,2.00,165.0,-3.3,-1.9,79.400000,28.95,0.0
1995-01-13,1995.0,1.0,13.0,4.99,156.0,-3.5,-2.2,82.000000,27.86,1.0
1995-07-09,1995.0,7.0,9.0,-4.97,-125.0,-8.2,1.5,84.200000,26.02,2.0
1996-07-17,1996.0,7.0,17.0,8.01,165.0,-5.4,-1.0,84.100000,28.38,3.0
1993-08-23,1993.0,8.0,23.0,2.06,165.0,2.0,0.6,75.400000,28.47,4.0
...,...,...,...,...,...,...,...,...,...,...
1996-03-04,1996.0,3.0,4.0,8.00,156.0,-6.3,-2.6,79.600000,27.58,26098.0
1994-11-25,1994.0,11.0,25.0,2.01,-179.9,-3.5,-2.6,78.300000,29.97,26099.0
1994-07-16,1994.0,7.0,16.0,5.00,-170.0,-4.7,1.2,76.600000,28.60,26100.0


*Sorting the dataframe*

In [15]:
sorted_df1 = train.sort_values(by=['year','month','day'])
sorted_df2 = evaluation.sort_values(by=['year','month','day'])

In [16]:
sorted_df1

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.,Index
Date,,,,,,,,,,,
1980-03-08,1980.0,3.0,8.0,-0.02,-109.44,-4.900000,1.100000,90.097506,25.66,25.97,31504.0
1980-03-09,1980.0,3.0,9.0,-0.02,-109.44,-4.500000,2.200000,90.007572,25.69,25.28,91306.0
1980-03-10,1980.0,3.0,10.0,-0.02,-109.44,-3.800000,1.900000,90.263205,25.56,24.31,1058.0
1980-03-12,1980.0,3.0,12.0,-0.02,-109.44,-4.400000,0.300000,91.656427,24.72,23.64,85273.0
1980-03-13,1980.0,3.0,13.0,-0.02,-109.44,-3.200000,0.100000,92.136024,24.66,24.34,37210.0
...,...,...,...,...,...,...,...,...,...,...,...
1996-12-31,1996.0,12.0,31.0,-8.04,164.90,-0.788924,-0.095612,84.900000,28.16,29.45,101096.0
1996-12-31,1996.0,12.0,31.0,-0.02,-170.00,-6.200000,-0.900000,72.800000,26.66,27.31,101423.0
1996-12-31,1996.0,12.0,31.0,0.01,-179.90,-4.700000,-0.700000,77.500000,27.62,28.50,101749.0


In [17]:
sorted_df2

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,Index
Date,,,,,,,,,,
1980-03-07,1980.0,3.0,7.0,-0.02,-109.44,-6.800000,0.700000,88.578514,26.140000,4951.0
1980-03-11,1980.0,3.0,11.0,-0.02,-109.44,-4.200000,1.500000,90.447573,25.300000,16738.0
1980-08-27,1980.0,8.0,27.0,0.00,-109.56,-3.800000,3.700000,93.428394,22.360000,10752.0
1980-08-29,1980.0,8.0,29.0,0.00,-109.56,-5.000000,2.400000,93.136024,22.450000,5289.0
1980-08-31,1980.0,8.0,31.0,0.00,-109.56,-6.200000,5.100000,91.454445,23.120000,2392.0
...,...,...,...,...,...,...,...,...,...,...
1996-12-31,1996.0,12.0,31.0,5.04,-109.94,-3.615432,1.914974,87.900000,24.690000,19542.0
1996-12-31,1996.0,12.0,31.0,1.99,-170.00,-5.800000,-0.100000,74.700000,27.440000,22163.0
1996-12-31,1996.0,12.0,31.0,-5.02,-179.90,3.800000,3.500000,82.400000,27.380000,23173.0


*Separating the dataframe into train and test*

In [18]:
x_train = sorted_df1.drop(columns=['s.s.temp.','Index']).values
y_train = sorted_df1['s.s.temp.'].values
x_test = sorted_df2.drop(columns=['Index']).values

Using Random Forest Regressor to predict the Sea Surface Temperature

In [20]:
rf=RandomForestRegressor(n_estimators=100,max_depth=100,min_samples_split=20,min_samples_leaf=10,random_state=0)
model_rf1=rf.fit(x_train,y_train)
predrf=rf.predict(x_test)

In [21]:
sorted_df2['s.s.temp.']=predrf

In [22]:
sorted_df2

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,Index,s.s.temp.
Date,,,,,,,,,,,
1980-03-07,1980.0,3.0,7.0,-0.02,-109.44,-6.800000,0.700000,88.578514,26.140000,4951.0,26.264380
1980-03-11,1980.0,3.0,11.0,-0.02,-109.44,-4.200000,1.500000,90.447573,25.300000,16738.0,25.627247
1980-08-27,1980.0,8.0,27.0,0.00,-109.56,-3.800000,3.700000,93.428394,22.360000,10752.0,22.668034
1980-08-29,1980.0,8.0,29.0,0.00,-109.56,-5.000000,2.400000,93.136024,22.450000,5289.0,22.738672
1980-08-31,1980.0,8.0,31.0,0.00,-109.56,-6.200000,5.100000,91.454445,23.120000,2392.0,23.519858
...,...,...,...,...,...,...,...,...,...,...,...
1996-12-31,1996.0,12.0,31.0,5.04,-109.94,-3.615432,1.914974,87.900000,24.690000,19542.0,26.209274
1996-12-31,1996.0,12.0,31.0,1.99,-170.00,-5.800000,-0.100000,74.700000,27.440000,22163.0,28.218569
1996-12-31,1996.0,12.0,31.0,-5.02,-179.90,3.800000,3.500000,82.400000,27.380000,23173.0,29.596077


In [23]:
sorted_df2 = sorted_df2.sort_values(by=['Index'])
sorted_df2

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,Index,s.s.temp.
Date,,,,,,,,,,,
1995-06-22,1995.0,6.0,22.0,2.00,165.0,-3.3,-1.9,79.400000,28.95,0.0,29.643849
1995-01-13,1995.0,1.0,13.0,4.99,156.0,-3.5,-2.2,82.000000,27.86,1.0,28.957582
1995-07-09,1995.0,7.0,9.0,-4.97,-125.0,-8.2,1.5,84.200000,26.02,2.0,26.140376
1996-07-17,1996.0,7.0,17.0,8.01,165.0,-5.4,-1.0,84.100000,28.38,3.0,29.285683
1993-08-23,1993.0,8.0,23.0,2.06,165.0,2.0,0.6,75.400000,28.47,4.0,29.600013
...,...,...,...,...,...,...,...,...,...,...,...
1996-03-04,1996.0,3.0,4.0,8.00,156.0,-6.3,-2.6,79.600000,27.58,26098.0,28.569095
1994-11-25,1994.0,11.0,25.0,2.01,-179.9,-3.5,-2.6,78.300000,29.97,26099.0,29.809746
1994-07-16,1994.0,7.0,16.0,5.00,-170.0,-4.7,1.2,76.600000,28.60,26100.0,29.094926


In [24]:
sorted_df2.drop(columns=['Index'],inplace=True)
sorted_df2.reset_index(inplace=True)
sorted_df2

,Date,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.
0,1995-06-22,1995.0,6.0,22.0,2.00,165.0,-3.3,-1.9,79.400000,28.95,29.643849
1,1995-01-13,1995.0,1.0,13.0,4.99,156.0,-3.5,-2.2,82.000000,27.86,28.957582
2,1995-07-09,1995.0,7.0,9.0,-4.97,-125.0,-8.2,1.5,84.200000,26.02,26.140376
3,1996-07-17,1996.0,7.0,17.0,8.01,165.0,-5.4,-1.0,84.100000,28.38,29.285683
4,1993-08-23,1993.0,8.0,23.0,2.06,165.0,2.0,0.6,75.400000,28.47,29.600013
...,...,...,...,...,...,...,...,...,...,...,...
26098,1996-03-04,1996.0,3.0,4.0,8.00,156.0,-6.3,-2.6,79.600000,27.58,28.569095
26099,1994-11-25,1994.0,11.0,25.0,2.01,-179.9,-3.5,-2.6,78.300000,29.97,29.809746
26100,1994-07-16,1994.0,7.0,16.0,5.00,-170.0,-4.7,1.2,76.600000,28.60,29.094926
26101,1993-07-12,1993.0,7.0,12.0,-0.04,-170.0,-1.6,2.3,80.600000,28.66,29.518449


In [25]:
sorted_df2.drop(columns=['Date'],inplace=True)
sorted_df2

,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.
0,1995.0,6.0,22.0,2.00,165.0,-3.3,-1.9,79.400000,28.95,29.643849
1,1995.0,1.0,13.0,4.99,156.0,-3.5,-2.2,82.000000,27.86,28.957582
2,1995.0,7.0,9.0,-4.97,-125.0,-8.2,1.5,84.200000,26.02,26.140376
3,1996.0,7.0,17.0,8.01,165.0,-5.4,-1.0,84.100000,28.38,29.285683
4,1993.0,8.0,23.0,2.06,165.0,2.0,0.6,75.400000,28.47,29.600013
...,...,...,...,...,...,...,...,...,...,...
26098,1996.0,3.0,4.0,8.00,156.0,-6.3,-2.6,79.600000,27.58,28.569095
26099,1994.0,11.0,25.0,2.01,-179.9,-3.5,-2.6,78.300000,29.97,29.809746
26100,1994.0,7.0,16.0,5.00,-170.0,-4.7,1.2,76.600000,28.60,29.094926
26101,1993.0,7.0,12.0,-0.04,-170.0,-1.6,2.3,80.600000,28.66,29.518449


In [26]:
sorted_df2.to_csv('evaluation_filled_sst.csv',index=True)